# IF702 Redes Neurais
Este notebook contém um script base para o projeto da disciplina IF702 Redes Neurais.

In [48]:
import numpy as np
import pandas as pd
import itertools

from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import EarlyStopping

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.cross_validation import PredefinedSplit
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
from sklearn.metrics import roc_auc_score, average_precision_score
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier

from imblearn.over_sampling import SMOTE

import matplotlib
matplotlib.use('nbagg')
import matplotlib.pyplot as plt

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


## Leitura e Limpeza dos Dados

A leitura dos dados é feita utilizando a biblioteca `pandas`. O presente exemplo importa a base de dados `mammography`. Caso você esteja trabalhando com outro data set, modifique este trecho de código.
Para importar o conjunto de dados do PAKDD, use a função `pd.read_table` ao invés da `pd.read_csv`.

In [5]:
data_set = pd.read_table('data/TRN')
data_set.drop_duplicates(inplace=True)  # Remove exemplos repetidos

In [6]:
# Exibe as 5 primeiras linhas do data set
data_set.head(5)

,INDEX,UF_1,UF_2,UF_3,UF_4,UF_5,UF_6,UF_7,IDADE,SEXO_1,...,CEP4_7,CEP4_8,CEP4_9,CEP4_10,CEP4_11,CEP4_12,CEP4_13,CEP4_14,IND_BOM_1_1,IND_BOM_1_2
0,0,1,1,1,0,0,0,0,0.135098,1,...,0,0,1,1,0,1,1,1,0,1
1,1,1,0,1,0,0,1,0,0.273504,1,...,0,1,0,1,1,0,0,0,1,0
2,2,1,0,1,0,0,1,0,0.281910,0,...,1,1,0,0,0,0,1,0,1,0
3,3,1,1,1,0,0,0,0,0.225741,0,...,1,1,0,1,1,0,1,0,1,0
4,4,1,1,0,0,0,1,0,0.480403,0,...,1,1,1,0,0,1,0,1,1,0


In [7]:
# Estatísticas sobre as variáveis
data_set.describe()

,INDEX,UF_1,UF_2,UF_3,UF_4,UF_5,UF_6,UF_7,IDADE,SEXO_1,...,CEP4_7,CEP4_8,CEP4_9,CEP4_10,CEP4_11,CEP4_12,CEP4_13,CEP4_14,IND_BOM_1_1,IND_BOM_1_2
count,389196.00000,389196.000000,389196.000000,389196.000000,389196.000000,389196.000000,389196.000000,389196.000000,3.891960e+05,389196.000000,...,389196.000000,389196.000000,389196.000000,389196.000000,389196.000000,389196.000000,389196.000000,389196.000000,389196.000000,389196.000000
mean,194597.50000,0.889274,0.691952,0.476552,0.296195,0.241179,0.218011,0.186836,4.552049e-01,0.521514,...,0.423378,0.417540,0.425708,0.459820,0.440842,0.436896,0.433709,0.440339,0.655449,0.344551
std,112351.35202,0.313793,0.461687,0.499451,0.456579,0.427799,0.412895,0.389781,2.537459e-01,0.499538,...,0.494095,0.493154,0.494451,0.498384,0.496489,0.496002,0.495587,0.496428,0.475222,0.475222
min,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.506237e-16,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,97298.75000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.507866e-01,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,194597.50000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4.375241e-01,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
75%,291896.25000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,6.578835e-01,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
max,389195.00000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000e+00,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


Agora vamos separar o data set em atributos dependentes (X = features) e independentes (y = classe). No caso do `mammography` a classe majoritária está codificada como -1 e a classe minoritária está codificada como 1. Para treinar nossa rede neural precisamos que os valores de classe sejam 0 e 1 (saída da última camada é uma sigmóide), assim modificamos a codificação da majoritária para 0.

Perceba que esse pré-processamento varia de data set para data set.

In [8]:
X = data_set.drop(['INDEX', 'IND_BOM_1_1', 'IND_BOM_1_2'], axis=1)
y = data_set[['IND_BOM_1_1', 'IND_BOM_1_2']]

# IND_BOM_1_1 e IND_BOM_1_2 sao os indicadores de "bom pagador", o que se refere a conceder o credito
# ou nao. Ou seja, a resposta de uma vai ser o inverso da outra. 

# Vamos utilzizar apenas uma como target, pois ambas representam "a mesma coisa".

y = y.drop(['IND_BOM_1_2'], axis=1)

## Divisão dos Dados em Treino, Validação, e Teste

Aqui dividimos o data set em treino, validação e teste de maneira estratificada.

In [15]:
## Treino: 50%, Validação: 25%, Teste: 25%
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=1/4, 
                                                    random_state=42, shuffle=True, stratify=y)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=1/3, 
                                                  random_state=42, shuffle=True, stratify=y_train)

print(y_train['IND_BOM_1_1'].value_counts())

1    127549
0     67049
Name: IND_BOM_1_1, dtype: int64


## Sampling dos Dados e Normalização

Para testar o comportamento da rede com diferentes funções de sampling, as mesmas devem ser implementadas e aplicadas ao conjunto de treinamento antes da normalização dos dados (você também pode investigar qual o efeito de aplicar o sampling após a normalização).

In [16]:
## TO DO -- Implementar as funções de sampling a serem utilizadas

y_train_group = y_train.groupby('IND_BOM_1_1')
print(y_train_group.size())

X_train_columns = X_train.columns

# oversampling nos dados de treinamento
X_train, y_train = SMOTE().fit_sample(X_train, y_train)

y_train = pd.DataFrame(np.array(y_train), columns=['IND_BOM_1_1'])
X_train = pd.DataFrame(np.array(X_train), columns=X_train_columns)

print("Base de treinamento")
print(y_train['IND_BOM_1_1'].value_counts())

# oversampling nos dados de validação
#X_val, y_val = SMOTE().fit_sample(X_val, y_val)
#print("Base de validação")
#print("Class 0: ", np.where(y_val == 0)[0].shape[0])
#print("Class 1: ", np.where(y_val == 1)[0].shape[0])

#y_#val = pd.DataFrame(np.array(y_val), columns=['IND_BOM_1_1'])
#X_v#al = pd.DataFrame(np.array(X_val), columns=X_train_columns)


IND_BOM_1_1
0     67049
1    127549
dtype: int64


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Base de treinamento
1    127549
0    127549
Name: IND_BOM_1_1, dtype: int64


In [31]:
X_train = pd.DataFrame(np.array(X_train), columns=X_train_columns)
X_train.to_csv('X_train_sampling.csv', index=False)
y_train.to_csv('y_train_sampling.csv', index=False)

É importante lembrar de normalizar os dados. A classe `StandardScaler` centraliza as variáveis e transforma as features para terem variância unitária. Você pode testar outras opções como o `MinMaxScaler`.

Todas as alternativas estão disponíveis em:
http://scikit-learn.org/stable/modules/classes.html#module-sklearn.preprocessing.

In [17]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

## Definição e Treino da Rede

Aqui definimos a arquitetura da nossa rede neural e treinamos ela.

No presente exemplo a rede possui apenas uma camada escondida. O código é bem intuitivo e a adição de novas camadas pode ser feita através da função `add`.

Para treinar a rede várias funções de otimização estão disponíveis. 

Confira os exemplos em: https://keras.io/optimizers/

O treinamento da rede pode ser interrompido baseado na performance dela em um conjunto de validação através de callbacks.

Confira a documentação da classe `EarlyStopping`: https://keras.io/callbacks/

In [9]:
# Número de features do nosso data set.
input_dim = X_train.shape[1]

# Aqui criamos o esboço da rede.
classifier = Sequential()

# Agora adicionamos a primeira camada escondida contendo 16 neurônios e função de ativação
# tangente hiperbólica. Por ser a primeira camada adicionada à rede, precisamos especificar
# a dimensão de entrada (número de features do data set).
classifier.add(Dense(16, activation='tanh', input_dim=input_dim))

# Em seguida adicionamos a camada de saída. Como nosso problema é binário só precisamos de
# 1 neurônio com função de ativação sigmoidal. A partir da segunda camada adicionada keras já
# consegue inferir o número de neurônios de entrada (16) e nós não precisamos mais especificar.
classifier.add(Dense(1, activation='sigmoid'))

# Por fim compilamos o modelo especificando um otimizador, a função de custo, e opcionalmente
# métricas para serem observadas durante treinamento.
classifier.compile(optimizer='adam', loss='mean_squared_error')

In [10]:
# Para treinar a rede passamos o conjunto de treinamento e especificamos o tamanho do mini-batch,
# o número máximo de épocas, e opcionalmente callbacks. No seguinte exemplo utilizamos early
# stopping para interromper o treinamento caso a performance não melhore em um conjunto de validação.
history = classifier.fit(X_train, y_train, batch_size=64, epochs=100000, 
                         callbacks=[EarlyStopping(patience=3)], validation_data=(X_val, y_val))

Train on 3924 samples, validate on 1962 samples
Epoch 1/100000
3924/3924 [==============================] - 1s 369us/step - loss: 0.2219 - val_loss: 0.1736
Epoch 2/100000
3924/3924 [==============================] - 0s 38us/step - loss: 0.1418 - val_loss: 0.1149
Epoch 3/100000
3924/3924 [==============================] - 0s 38us/step - loss: 0.0925 - val_loss: 0.0745
Epoch 4/100000
3924/3924 [==============================] - 0s 37us/step - loss: 0.0603 - val_loss: 0.0506
Epoch 5/100000
3924/3924 [==============================] - 0s 36us/step - loss: 0.0423 - val_loss: 0.0378
Epoch 6/100000
3924/3924 [==============================] - 0s 32us/step - loss: 0.0327 - val_loss: 0.0310
Epoch 7/100000
3924/3924 [==============================] - 0s 37us/step - loss: 0.0274 - val_loss: 0.0271
Epoch 8/100000
3924/3924 [==============================] - 0s 36us/step - loss: 0.0242 - val_loss: 0.0248
Epoch 9/100000
3924/3924 [==============================] - 0s 39us/step - loss: 0.0222 - val_l

In [18]:
classifiers = []
forest = RandomForestClassifier()
forest.fit(X_train, y_train)

classifiers.append(('Random Forest', forest))

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


In [35]:
svm = SVC()
svm.fit(X_train, y_train)

classifiers.append(('SVM', svm))

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [50]:
g_boosting = GradientBoostingClassifier()
g_boosting.fit(X_train, y_train)

classifiers.append(('Gradient Boosting', g_boosting))

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [32]:
mlp = MLPClassifier()
mlp.fit(X_train, y_train)

classifiers.append(('MLP', mlp))

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:912: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [36]:
for classfier in classifiers:
    print(classfier[0])
    y_pred_val = classfier[1].predict(X_val)
    y_pred_val_scores =  classfier[1].predict_proba(X_val)[:, 1]
    accuracy, recall, precision, f1, auroc, aupr = compute_performance_metrics(y_val, y_pred_val, y_pred_val_scores)
    print_metrics_summary(accuracy, recall, precision, f1, auroc, aupr)
    print('\n')

Random Forest

Accuracy:         0.6242
Recall:           0.6937
Precision:        0.7220
F1:               0.7076
AUROC:            0.6364
AUPR:             0.7506


MLP

Accuracy:         0.6277
Recall:           0.7216
Precision:        0.7137
F1:               0.7176
AUROC:            0.6297
AUPR:             0.7526


SVM


AttributeError: predict_proba is not available when  probability=False

In [49]:
experiments_params = {'hidden_layer_sizes' : [(100,), (50,), (100, 25, 10)], 'activation' : ['logistic', 'tanh', 'relu'], 'solver' : ['sgd'], 'learning_rate_init' : [0.001, 0.01, 0.1]}

keys, values = zip(*experiments_params.items())
for v in itertools.product(*values):
    params = dict(zip(keys, v))
    print(params)
    
    mlp = MLPClassifier(**params)
    mlp.fit(X_train, y_train)
    
    y_pred_val = mlp.predict(X_val)
    y_pred_val_scores =  mlp.predict_proba(X_val)[:, 1]
    accuracy, recall, precision, f1, auroc, aupr = compute_performance_metrics(y_val, y_pred_val, y_pred_val_scores)
    print_metrics_summary(accuracy, recall, precision, f1, auroc, aupr)

{'hidden_layer_sizes': (100,), 'activation': 'logistic', 'solver': 'sgd', 'learning_rate_init': 0.001}


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:912: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)



Accuracy:         0.6279
Recall:           0.6234
Precision:        0.7654
F1:               0.6871
AUROC:            0.6777
AUPR:             0.7881
{'hidden_layer_sizes': (100,), 'activation': 'logistic', 'solver': 'sgd', 'learning_rate_init': 0.01}


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)



Accuracy:         0.6174
Recall:           0.6890
Precision:        0.7164
F1:               0.7024
AUROC:            0.6245
AUPR:             0.7485
{'hidden_layer_sizes': (100,), 'activation': 'logistic', 'solver': 'sgd', 'learning_rate_init': 0.1}

Accuracy:         0.6239
Recall:           0.7178
Precision:        0.7111
F1:               0.7144
AUROC:            0.6239
AUPR:             0.7490
{'hidden_layer_sizes': (100,), 'activation': 'tanh', 'solver': 'sgd', 'learning_rate_init': 0.001}

Accuracy:         0.6234
Recall:           0.6796
Precision:        0.7278
F1:               0.7029
AUROC:            0.6406
AUPR:             0.7607
{'hidden_layer_sizes': (100,), 'activation': 'tanh', 'solver': 'sgd', 'learning_rate_init': 0.01}

Accuracy:         0.6126
Recall:           0.6863
Precision:        0.7122
F1:               0.6990
AUROC:            0.6145
AUPR:             0.7411
{'hidden_layer_sizes': (100,), 'activation': 'tanh', 'solver': 'sgd', 'learning_rate_init': 0.1}



Algumas funções auxiliares.

In [20]:
def extract_final_losses(history):
    """Função para extrair o melhor loss de treino e validação.
    
    Argumento(s):
    history -- Objeto retornado pela função fit do keras.
    
    Retorno:
    Dicionário contendo o melhor loss de treino e de validação baseado 
    no menor loss de validação.
    """
    train_loss = history.history['loss']
    val_loss = history.history['val_loss']
    idx_min_val_loss = np.argmin(val_loss)
    return {'train_loss': train_loss[idx_min_val_loss], 'val_loss': val_loss[idx_min_val_loss]}

def plot_training_error_curves(history):
    """Função para plotar as curvas de erro do treinamento da rede neural.
    
    Argumento(s):
    history -- Objeto retornado pela função fit do keras.
    
    Retorno:
    A função gera o gráfico do treino da rede e retorna None.
    """
    train_loss = history.history['loss']
    val_loss = history.history['val_loss']
    
    fig, ax = plt.subplots()
    ax.plot(train_loss, label='Train')
    ax.plot(val_loss, label='Validation')
    ax.set(title='Training and Validation Error Curves', xlabel='Epochs', ylabel='Loss (MSE)')
    ax.legend()
    plt.show()

def compute_performance_metrics(y, y_pred_class, y_pred_scores=None):
    accuracy = accuracy_score(y, y_pred_class)
    recall = recall_score(y, y_pred_class)
    precision = precision_score(y, y_pred_class)
    f1 = f1_score(y, y_pred_class)
    performance_metrics = (accuracy, recall, precision, f1)
    if y_pred_scores is not None:
        auroc = roc_auc_score(y, y_pred_scores)
        aupr = average_precision_score(y, y_pred_scores)
        performance_metrics = performance_metrics + (auroc, aupr)
    return performance_metrics

def print_metrics_summary(accuracy, recall, precision, f1, auroc=None, aupr=None):
    print()
    print("{metric:<18}{value:.4f}".format(metric="Accuracy:", value=accuracy))
    print("{metric:<18}{value:.4f}".format(metric="Recall:", value=recall))
    print("{metric:<18}{value:.4f}".format(metric="Precision:", value=precision))
    print("{metric:<18}{value:.4f}".format(metric="F1:", value=f1))
    if auroc is not None:
        print("{metric:<18}{value:.4f}".format(metric="AUROC:", value=auroc))
    if aupr is not None:
        print("{metric:<18}{value:.4f}".format(metric="AUPR:", value=aupr))

In [12]:
plot_training_error_curves(history)

<IPython.core.display.Javascript object>